# ceph-ansible安装ceph

参考：https://blog.csdn.net/junming_zhao/article/details/87160900

## 准备工作

### 准备机器
+ 1台 ceph-deploy，需安装ansible
    - 192.168.20.3 wfh_deploy
+ 3台 ceph-server，安装基本服务
    - 192.168.20.204 wfh_node04
    - 192.168.20.205 wfh_node05
    - 192.168.20.206 wfh_node06
+ 1台 ceph-client，测试用
    - 192.168.20.201 wfh_node01

### 在 ceph-deploy 上添加主机名和IP映射
在此主机的 /etc/hosts 上应包括类似下面的信息：
```ini
192.168.20.3 wfh_deploy
192.168.20.204 wfh_node04
192.168.20.205 wfh_node05
192.168.20.206 wfh_node06
```

### 添加ssh免密码登录

从 ceph-deploy 到 ceph-server 和 ceph-client 能够通过 ssh 直接登录，无需密码。

+ 这里使用root用户。
+ 具体方法（:TODO）

### 配置时间同步服务
本集群统一采用 chrony 代替传统 ntp 服务。

### 添加 ceph 的 repo源，选择安装rpm-nautilus版本
```bash
# 添加 ceph的 aliyun源
# [root@wfh_deploy ~]#

cat > ceph-rpm-nautilus.sh<<EOF
rpm --import https://mirrors.aliyun.com/centos/RPM-GPG-KEY-CentOS-7
curl -o /etc/yum.repos.d/epel.repo -OL http://mirrors.aliyun.com/repo/epel-7.repo

yum -y install epel-release yum-plugin-priorities \\
    https://mirrors.aliyun.com/ceph/rpm-nautilus/el7/noarch/ceph-release-1-1.el7.noarch.rpm
sed -i -e "s/enabled=1/enabled=1\npriority=1/g" /etc/yum.repos.d/ceph.repo
sed -i 's/download.ceph.com/mirrors.aliyun.com\/ceph/g' /etc/yum.repos.d/ceph.repo

yum update -y
EOF

cat > deploy-ceph-repos.sh<<EOF
chmod +x ceph-rpm-nautilus.sh
for num in {1,2,3,4,5,6}
do
    scp ceph-rpm-nautilus.sh wfh_node0\$num:~/
    ssh wfh_node0\$num "~/ceph-rpm-nautilus.sh && rm -f ~/ceph-rpm-nautilus.sh"
done
rm -f ceph-rpm-nautilus.sh
EOF
chmod +x deploy-ceph-repos.sh

./deploy-ceph-repos.sh
```

## 安装和配置 ansible

### 安装 ansible
```bash
# for Centos7
sudo yum install -y ansible
```

### 编排 hosts
在 /etc/ansible/hosts 文件里增加下面节点

```ini
# 增加ceph服务监视节点
[mons]
wfh_node04
wfh_node05
wfh_node06

# 增加ceph服务osd节点
[osds]
wfh_node04
wfh_node05
wfh_node06

# 增加ceph管理节点
[mgrs]
wfh_node01
wfh_node02
```

### 编排 group
```bash
#[root@wfh_deploy ceph-ansible-3.2.27]# 
cp group_vars/all.yml{.sample,}
```